# Bragg-edge imaging with ODIN

This notebook illustrates how to convert recorded events on the ODIN detector to a single wavelength spectrum,
revealing a Bragg edge in the data.
WFM mode was used in the chopper cascade.

In [ ]:
import scipp as sc
from ess.reduce import time_of_flight
from ess import odin
import ess.odin.data  # noqa: F401
from ess.imaging.types import *

## Create and configure the workflow

In [ ]:
wf = odin.OdinBraggEdgeWorkflow()

wf[Filename[SampleRun]] = odin.data.iron_simulation_sample_small()
wf[NeXusDetectorName] = "event_mode_detectors/timepix3"
wf[time_of_flight.TimeOfFlightLookupTableFilename] = odin.data.odin_tof_lookup_table()

## First look at the data

We load the raw detector data and perform a quick visualization of the `event_time_offset` spectrum.

In [ ]:
tmpx3 = wf.compute(DetectorData[SampleRun])
tmpx3

In [ ]:
tmpx3.bins.concat().hist(event_time_offset=300).plot()

## Compute neutron time-of-flight/wavelength

We will now use the workflow to compute the neutron time-of-flight (equivalent to wavelength) using a lookup table built from the beamline chopper information.

In [ ]:
wf.visualize(DetectorTofData[SampleRun], graph_attr={"rankdir": "LR"})

### Inspect the lookup table

It is always a good idea to quickly plot the TOF lookup table, as a sanity check.

In [ ]:
table = wf.compute(time_of_flight.TimeOfFlightLookupTable)
table.plot(figsize=(9, 4))

### Compute neutron wavelengths

In [ ]:
sample_wavs = wf.compute(CountsWavelength[SampleRun])

sample_wavs.bins.concat().hist(wavelength=300).plot()

## Process the open-beam run

We now reuse the same workflow to process the open-beam run that will be used later for normalization.

In [ ]:
wf[Filename[OpenBeamRun]] = odin.data.iron_simulation_ob_small()
openbeam_wavs = wf.compute(CountsWavelength[OpenBeamRun])

openbeam_wavs.bins.concat().hist(wavelength=300).plot()

## Select region of interest by masking outer regions

Making a 2D histogram of the data shows a dark square region in the centre of the detector panel;
this is the region of interest, where the square sample has absorbed neutrons.

In [ ]:
sample_wavs.hist().plot(aspect='equal')

The brighter areas around the edges are regions where neutrons did not travel through the sample.
We thus want to mask those out using masking rules based on the spatial coordinates of the data:

In [ ]:
wf[MaskingRules] = {
    'x_pixel_offset': lambda x: (x < sc.scalar(-5.8e-3, unit='m').to(unit=x.unit)) | (x > sc.scalar(5.8e-3, unit='m').to(unit=x.unit)),
    'y_pixel_offset': lambda y: (y < sc.scalar(-5.8e-3, unit='m').to(unit=y.unit)) | (y > sc.scalar(5.8e-3, unit='m').to(unit=y.unit))
}

In [ ]:
masked = wf.compute(CountsMasked[SampleRun])

masked.hist().plot(aspect='equal')

## Normalize to open beam

Finally, we use the masked sample and open-beam data to obtain a normalized signal,
which reveals the Fe Bragg edges:

In [ ]:
wbins = sc.linspace('wavelength', 1.1, 9.4, 301, unit='angstrom')

normalized = (
    wf.compute(CountsMasked[SampleRun]).bins.concat().hist(wavelength=wbins) /
    wf.compute(CountsMasked[OpenBeamRun]).bins.concat().hist(wavelength=wbins)
)

normalized.plot()

## Save the final result

In [ ]:
from scippneutron.io import save_xye

to_disk = normalized.copy(deep=False)
to_disk.coords['wavelength'] = sc.midpoints(to_disk.coords['wavelength'])

save_xye('fe_bragg_edge.xye', to_disk)